In [1]:
library("tidyverse")
players = read_csv("data/players.csv")
sessions = read_csv("data/sessions.csv")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.2
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


### Data Description
##### sessions.csv
There were 1535 observations made in the sessions dataset meaning there was a total of 1535 sessions played. 
There are five varibles in sessions.csv: 
- hashedEmail: chr
    - Serves as a player identifyer between both datasets
    - The email with the most sessions played had a total of 310 sessions
- start_time: chr
    - String representaion of start time
- end_time: chr
    - String representaion of end time
- original_start_time: dbl
    - Number representation of start time
- original_end_time: dbl
    - Number representation of end time
    - The longest single session was four hours long
##### players.csv
There were 196 observations in the players dataset meaning there was 196 unique players. 
In players.csv there seven varibles: 
- experience: chr
    - A catagorical varible denoting the experience level of the player
    - 63 players were Amateurs, 48 were Veterans, 36 were Regular, 35 were Beginners, and 14 were pros
- subscribe: lgl
    - A boolean value representing whether or not a player subscribed
    - Most players subscribed with 144 subscribing, and 52 not subscribing
- hashedEmail: chr
    - As previously mentioned: a unique identifyer given to each player to track sessions
- played_hours: dbl
    - Total hours played by each player
    - Most hours played was 223.1 while the least was 0
- name: chr
    - The name of the player
- gender: chr
    - The players gender
    - 124 Males, 37 Females, 15 Non-Binary, 11 Perfer not to say, 6 Two-Spirited, 2 Agender, and 1 Other
- age: dbl
    - The players age
    - The oldest player was 58 and the youngest was 9
##### Issues
I dont directly see any major issues in the data. 
- The name column in players.csv is somewhat irrelavent as majority of the names are unique and that isnt representative of anything. It would be good to move the hashedEmail column to the first position in the dataset for readability. 
- Sessions.csv also has both a string and number representation of start and end times. A better representation would be to keep the string columns for start and end, and replace the number columns with a single column representing the total session time. This makes me belive it is raw data collected from the database as the number timestamps would be what was outputted automatically, and then reformatted as a string.